In [ ]:
from pyspark.sql import SparkSession
from IPython.display import display
import pyspark.sql.functions as f
from pyspark.ml.feature import Word2Vec
import datetime
import numpy as np

%precision 3

'%.3f'

In [2]:
spark = SparkSession.builder \
    .appName("Word2Vec") \
    .master("local[*]") \
    .getOrCreate()
    
# spark = SparkSession.builder \
#     .appName("Word2Vec") \
#     .master("spark://raspberrypi-dml0:7077") \
#     .config("spark.cores.max", "16") \
#     .config("spark.executor.cores", "4") \
#     .config("spark.executor.memory", "5500m") \
#     .config("spark.driver.memory", "4g") \
#     .config("spark.memory.fraction", "0.8") \
#     .getOrCreate()
    
print(f"Master: {spark.sparkContext.master}")
print(f"App Name: {spark.sparkContext.appName}")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/25 20:53:28 WARN Utils: Your hostname, mahdi-laptop, resolves to a loopback address: 127.0.1.1; using 192.168.0.95 instead (on interface wlo1)
25/12/25 20:53:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/25 20:53:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Master: local[*]
App Name: Word2Vec


In [3]:
# raw_data = spark.read.text('hdfs://raspberrypi-dml0:9000/vajhi/wiki_corpus')
raw_data = spark.read.text('wiki_corpus')
raw_data.show(5,50)

+--------------------------------------------------+
|                                             value|
+--------------------------------------------------+
|was reported before the land was made inaccessi...|
|genoese may refer to a person from genoa the ge...|
|one one nine seven seven in los angeles califor...|
|whose works deal with the survival side of life...|
|an attempted rape raven oak strove to free hers...|
+--------------------------------------------------+
only showing top 5 rows


In [4]:
split_data = raw_data.select(f.split(f.col('value'), ' ').alias('value'))
split_data.show(5, 50)

+--------------------------------------------------+
|                                             value|
+--------------------------------------------------+
|[was, reported, before, the, land, was, made, i...|
|[genoese, may, refer, to, a, person, from, geno...|
|[one, one, nine, seven, seven, in, los, angeles...|
|[whose, works, deal, with, the, survival, side,...|
|[an, attempted, rape, raven, oak, strove, to, f...|
+--------------------------------------------------+
only showing top 5 rows


In [5]:
vectorSize=200
windowSize=5
minCount=3
numPartitions=11
maxIter=20
w2v = Word2Vec(
    seed=42,
    inputCol="value",
    outputCol="model",
    
    vectorSize=vectorSize,
    windowSize=windowSize,
    minCount=minCount,
    numPartitions=numPartitions,
    maxIter=maxIter
    )

In [6]:
model = w2v.fit(split_data)

25/12/25 20:56:28 WARN BlockManager: Asked to remove block broadcast_19_piece11, which does not exist
25/12/25 20:56:28 WARN BlockManager: Asked to remove block broadcast_19_piece9, which does not exist
25/12/25 20:56:28 WARN BlockManager: Asked to remove block broadcast_19_piece13, which does not exist
25/12/25 20:56:29 WARN BlockManager: Asked to remove block broadcast_19_piece14, which does not exist
25/12/25 20:56:29 WARN BlockManager: Asked to remove block broadcast_19_piece1, which does not exist


In [7]:
# model.save(f'hdfs://raspberrypi-dml0:9000/vajhi/word2vec_VS{vectorSize}_WS{windowSize}_MC{minCount}_MI{maxIter}_{datetime.datetime.now().strftime("D%Y_%m_%d_T%H_%M")}')
model.save(f'word2vec_VS{vectorSize}_WS{windowSize}_MC{minCount}_MI{maxIter}_{datetime.datetime.now().strftime("D%Y_%m_%d_T%H_%M")}')

25/12/25 21:11:38 WARN TaskSetManager: Stage 65 contains a task of very large size (5780 KiB). The maximum recommended task size is 1000 KiB.


In [67]:
selected_words = ['iran', 'tehran', 'learning', 'science']
model.getVectors().filter(f.col('word').isin(selected_words)).show(truncate=100)

+--------+----------------------------------------------------------------------------------------------------+
|    word|                                                                                              vector|
+--------+----------------------------------------------------------------------------------------------------+
|    iran|[0.2611589729785919,-0.26786503195762634,-0.21497038006782532,0.1693626195192337,-0.1699647605419...|
| science|[0.1457558423280716,0.18273581564426422,0.15451723337173462,-0.15501490235328674,0.11617875099182...|
|  tehran|[0.5012955665588379,-0.03322823718190193,-0.0753512904047966,0.5106785893440247,-0.22006152570247...|
|learning|[0.021161837503314018,0.44284340739250183,0.08777477592229843,-0.3333715796470642,0.2834542393684...|
+--------+----------------------------------------------------------------------------------------------------+



In [43]:
for i in selected_words:
    print(i)
    model.findSynonyms(i, 5).join(model.getVectors(), 'word').show(5, 100)

iran


25/12/25 21:51:06 WARN TaskSetManager: Stage 101 contains a task of very large size (11232 KiB). The maximum recommended task size is 1000 KiB.
25/12/25 21:51:06 WARN TaskSetManager: Stage 102 contains a task of very large size (11234 KiB). The maximum recommended task size is 1000 KiB.
25/12/25 21:51:06 WARN TaskSetManager: Stage 103 contains a task of very large size (11234 KiB). The maximum recommended task size is 1000 KiB.


+-----------+------------------+----------------------------------------------------------------------------------------------------+
|       word|        similarity|                                                                                              vector|
+-----------+------------------+----------------------------------------------------------------------------------------------------+
|   pakistan|0.5902189016342163|[0.01546500250697136,-0.18069183826446533,-0.08781564235687256,0.19112782180309296,-0.08888671547...|
|    iranian| 0.478732705116272|[0.16548150777816772,0.03301917761564255,-0.16876323521137238,-0.3054888844490051,0.1121090352535...|
|       iraq|0.5148853063583374|[0.061682265251874924,-0.16518108546733856,-0.07135281711816788,0.01638515666127205,0.02287589013...|
|      syria|0.5015237927436829|[0.13604576885700226,-0.6059649586677551,-0.17501457035541534,-0.028692834079265594,0.24407251179...|
|afghanistan|0.5801478624343872|[0.20171602070331573,-0.243650

25/12/25 21:51:07 WARN TaskSetManager: Stage 105 contains a task of very large size (11232 KiB). The maximum recommended task size is 1000 KiB.
25/12/25 21:51:07 WARN TaskSetManager: Stage 106 contains a task of very large size (11234 KiB). The maximum recommended task size is 1000 KiB.
25/12/25 21:51:07 WARN TaskSetManager: Stage 107 contains a task of very large size (11234 KiB). The maximum recommended task size is 1000 KiB.


+----------+-------------------+----------------------------------------------------------------------------------------------------+
|      word|         similarity|                                                                                              vector|
+----------+-------------------+----------------------------------------------------------------------------------------------------+
|    jeddah|0.47724097967147827|[0.47635170817375183,0.02784269116818905,-0.24967725574970245,-0.2532123029232025,-0.372087478637...|
|     accra|0.44458678364753723|[0.39662715792655945,-0.3197892904281616,-0.2616196870803833,0.45456260442733765,-0.0626651197671...|
|  istanbul|0.44995713233947754|[0.12118062376976013,0.15797819197177887,-0.5601809024810791,0.5034356713294983,0.234818816184997...|
|dusseldorf| 0.4523930847644806|[0.15115147829055786,0.09853902459144592,0.038076236844062805,0.22718612849712372,-0.799529492855...|
|  chisinau|0.44119977951049805|[-0.09246578812599182,0.158015

25/12/25 21:51:08 WARN TaskSetManager: Stage 109 contains a task of very large size (11232 KiB). The maximum recommended task size is 1000 KiB.
25/12/25 21:51:08 WARN TaskSetManager: Stage 110 contains a task of very large size (11234 KiB). The maximum recommended task size is 1000 KiB.
25/12/25 21:51:08 WARN TaskSetManager: Stage 111 contains a task of very large size (11234 KiB). The maximum recommended task size is 1000 KiB.


+-----------+-------------------+----------------------------------------------------------------------------------------------------+
|       word|         similarity|                                                                                              vector|
+-----------+-------------------+----------------------------------------------------------------------------------------------------+
| vocational|0.43295395374298096|[0.20496653020381927,0.5086355805397034,-0.17237940430641174,-0.666351318359375,0.341406106948852...|
| montessori| 0.4428502917289734|[-0.07003387808799744,0.40951278805732727,-0.04785775765776634,-0.6152541041374207,0.095848195254...|
|educational| 0.4987924098968506|[-0.046458207070827484,0.3235059082508087,-0.172844797372818,-0.2934342324733734,0.35010048747062...|
|  education| 0.4539998471736908|[-0.15238501131534576,0.2325889766216278,0.01778140664100647,-0.5715274810791016,0.08364238590002...|
|     skills|0.47505220770835876|[-0.1114797443151474,0

25/12/25 21:51:09 WARN TaskSetManager: Stage 113 contains a task of very large size (11232 KiB). The maximum recommended task size is 1000 KiB.
25/12/25 21:51:09 WARN TaskSetManager: Stage 114 contains a task of very large size (11234 KiB). The maximum recommended task size is 1000 KiB.
25/12/25 21:51:09 WARN TaskSetManager: Stage 115 contains a task of very large size (11234 KiB). The maximum recommended task size is 1000 KiB.


+------------+------------------+----------------------------------------------------------------------------------------------------+
|        word|        similarity|                                                                                              vector|
+------------+------------------+----------------------------------------------------------------------------------------------------+
|anthropology|0.5277754664421082|[0.4610588550567627,0.2128603160381317,0.3301967978477478,-0.15136365592479706,-0.325629711151123...|
| mathematics|0.5373182892799377|[0.18220581114292145,-0.060965120792388916,-0.14754094183444977,-0.12785378098487854,-0.258761227...|
|    sciences| 0.608418345451355|[-0.1454285830259323,0.0399218387901783,-0.20353825390338898,-0.43593719601631165,-0.350875884294...|
|  humanities|0.5414108633995056|[0.0665922611951828,0.13013219833374023,-0.4057573974132538,-0.5677540898323059,-0.20385284721851...|
|     fiction|0.6225622892379761|[0.39104875922203064,0

In [63]:
def vac_calc(model, pos_words, neg_words):
    all_vec = model.getVectors().filter(f.col('word').isin(pos_words + neg_words)).collect()
    all_vec = {i.word: i.vector for i in all_vec}
    
    result = np.zeros(model.getVectorSize())
    
    for i in pos_words:
        result += all_vec[i]
    
    for i in neg_words:
        result -= all_vec[i]
        
    return result

In [ ]:
pos_words = ['king', 'woman']
neg_words = ['man']

model.findSynonyms(vac_calc(model, pos_words, neg_words), 5).show()
model.findSynonyms('king', 5).show()

+-------+-------------------+
|   word|         similarity|
+-------+-------------------+
|   king| 0.7492444515228271|
|  queen| 0.4900623857975006|
|amalric|  0.485809862613678|
|  elara| 0.4716344475746155|
|  woman|0.47139328718185425|
+-------+-------------------+

+-------+------------------+
|   word|        similarity|
+-------+------------------+
| afonso|0.5781086683273315|
|     iv|0.5645919442176819|
|    iii|0.5424816608428955|
|mongkut|0.5390035510063171|
| youmiu|0.5387355089187622|
+-------+------------------+

